In [137]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime as dt
from datetime import timedelta
from darksky import forecast
key = "e202a3b89ed3182ce3d86552661df096" #darksky key
from sklearn.linear_model import LogisticRegression

In [138]:
# US fires data - has the cause of fire
# https://www.fs.usda.gov/rds/archive/catalog/RDS-2013-0009.4
# Metadata - https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.4/_metadata_RDS-2013-0009.4.html
us_fires1 = pd.read_csv("us_fires_1.csv")
us_fires2 = pd.read_csv("us_fires_2.csv")
us_fires3 = pd.read_csv("us_fires_3.csv")
us_fires4 = pd.read_csv("us_fires_4.csv")
us_fires5 = pd.read_csv("us_fires_5.csv")
us_fires6 = pd.read_csv("us_fires_6.csv")
us_fires7 = pd.read_csv("us_fires_7.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,11,12,14,15,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12,16,17,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,11,12,13,14,15,18,25,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interac

In [139]:
# Concatenate all us_fires data
pd.set_option('display.max_columns', None)
us_fires_all = pd.concat([us_fires1, us_fires2], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires3], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires4], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires5], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires6], ignore_index=True)
us_fires_all = pd.concat([us_fires_all, us_fires7], ignore_index=True)

In [140]:
us_fires_all.head()
us_fires_all.tail()

,objectid,fod_id,fpa_id,source_system_type,source_system,nwcg_reporting_agency,nwcg_reporting_unit_id,nwcg_reporting_unit_name,source_reporting_unit,source_reporting_unit_name,local_fire_report_id,local_incident_id,fire_code,fire_name,ics_209_incident_number,ics_209_name,mtbs_id,mtbs_fire_name,complex_name,fire_year,discovery_date,discovery_doy,discovery_time,stat_cause_code,stat_cause_descr,cont_date,cont_doy,cont_time,fire_size,fire_size_class,latitude,longitude,owner_code,owner_descr,state,county,fips_code,fips_name
1880460,1880461,300348000.0,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,591814,009371,NaN,ODESSA 2,NaN,NaN,NaN,NaN,NaN,2015,2015-09-26T00:00:00,269,1726.0,13,Missing/Undefined,2015-09-26T00:00:00,269.0,1843.0,0.01,A,40.481637,-122.389375,13,STATE OR PRIVATE,CA,NaN,NaN,NaN
1880461,1880462,300348000.0,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,569419,000366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-10-05T00:00:00,278,126.0,9,Miscellaneous,NaN,NaN,NaN,0.20,A,37.617619,-120.938570,12,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN
1880462,1880463,300348000.0,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,574245,000158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-05-02T00:00:00,122,2052.0,13,Missing/Undefined,NaN,NaN,NaN,0.10,A,37.617619,-120.938570,12,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN
1880463,1880464,300348000.0,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,570462,000380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2015-10-14T00:00:00,287,2309.0,13,Missing/Undefined,NaN,NaN,NaN,2.00,B,37.672235,-120.898356,12,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN
1880464,1880465,300348000.0,2015CAIRS26733926,NONFED,ST-CACDF,ST/C&L,USCABDU,San Bernardino Unit,CABDU,CDF - San Bernardino Unit,535436,003225,NaN,BARKER BL BIG_BEAR_LAKE_,NaN,NaN,NaN,NaN,NaN,2015,2015-03-14T00:00:00,73,2128.0,9,Miscellaneous,NaN,NaN,NaN,0.10,A,34.263217,-116.830950,13,STATE OR PRIVATE,CA,NaN,NaN,NaN


In [141]:
len(us_fires_all)

1880465

In [142]:
train = us_fires_all[['objectid',
                      'fod_id',
                      'fpa_id',
                      'fire_year',
                      'discovery_date',
                      'discovery_doy',
                      'stat_cause_code',
                      'stat_cause_descr',
                      'fire_size',
                      'fire_size_class',
                      'latitude',
                      'longitude',
                      'state']]
train.head()

,objectid,fod_id,fpa_id,fire_year,discovery_date,discovery_doy,stat_cause_code,stat_cause_descr,fire_size,fire_size_class,latitude,longitude,state
0,1,1.0,FS-1418826,2005,2005-02-02T00:00:00,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,CA
1,2,2.0,FS-1418827,2004,2004-05-12T00:00:00,133,1,Lightning,0.25,A,38.933056,-120.404444,CA
2,3,3.0,FS-1418835,2004,2004-05-31T00:00:00,152,5,Debris Burning,0.10,A,38.984167,-120.735556,CA
3,4,4.0,FS-1418845,2004,2004-06-28T00:00:00,180,1,Lightning,0.10,A,38.559167,-119.913333,CA
4,5,5.0,FS-1418847,2004,2004-06-28T00:00:00,180,1,Lightning,0.10,A,38.559167,-119.933056,CA


In [143]:
train.isnull().sum()

objectid            0
fod_id              0
fpa_id              0
fire_year           0
discovery_date      0
discovery_doy       0
stat_cause_code     0
stat_cause_descr    0
fire_size           0
fire_size_class     0
latitude            0
longitude           0
state               0
dtype: int64

In [144]:
# Regex extract date
train['discovery_date'] = train['discovery_date'].str.extract(r'(\d{4}-\d{2}-\d{2})')
train.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,objectid,fod_id,fpa_id,fire_year,discovery_date,discovery_doy,stat_cause_code,stat_cause_descr,fire_size,fire_size_class,latitude,longitude,state
0,1,1.0,FS-1418826,2005,2005-02-02,33,9,Miscellaneous,0.10,A,40.036944,-121.005833,CA
1,2,2.0,FS-1418827,2004,2004-05-12,133,1,Lightning,0.25,A,38.933056,-120.404444,CA
2,3,3.0,FS-1418835,2004,2004-05-31,152,5,Debris Burning,0.10,A,38.984167,-120.735556,CA
3,4,4.0,FS-1418845,2004,2004-06-28,180,1,Lightning,0.10,A,38.559167,-119.913333,CA
4,5,5.0,FS-1418847,2004,2004-06-28,180,1,Lightning,0.10,A,38.559167,-119.933056,CA


In [145]:
# Convert to timestamp
train['discovery_date'] = train['discovery_date'].apply(lambda x: dt.strptime(x, '%Y-%m-%d'))
type(train['discovery_date'][1])
# Convert back to string
# train['discovery_date'] = train['discovery_date'].astype(str)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


pandas._libs.tslibs.timestamps.Timestamp

In [146]:
# Consolidate years - overall dataset is 1992 - 2015
len(train)
train.loc[train['fire_year'].idxmax()]

objectid                        1789012
fod_id                        3.002e+08
fpa_id                       FS-6341424
fire_year                          2015
discovery_date      2015-08-14 00:00:00
discovery_doy                       226
stat_cause_code                       1
stat_cause_descr              Lightning
fire_size                           0.1
fire_size_class                       A
latitude                        45.9342
longitude                      -113.021
state                                MT
Name: 1789011, dtype: object

In [147]:
test_recent = train[(train['discovery_date'] > '2015-01-01') & (train['discovery_date'] < '2015-08-14')]
test = test_recent.sample(5000)

train_recent = train[(train['discovery_date'] > '2014-01-01') & (train['discovery_date'] < '2014-12-31')]
train = train_recent.sample(10000)

In [148]:
train = train.drop(columns = ['stat_cause_descr','fpa_id'])
test = test.drop(columns = ['stat_cause_descr', 'fpa_id'])

In [149]:
states = pd.get_dummies(train['state'])
states['fod_id'] = train['fod_id']

In [150]:
train = pd.merge(train, states, how = 'outer', on = 'fod_id')
train = train.drop(columns = ['state'])


In [102]:
test.head()

,objectid,fod_id,fire_year,discovery_date,discovery_doy,stat_cause_code,fire_size,fire_size_class,latitude,longitude,state
1798262,1798263,300211000.0,2015,2015-06-16,167,13,10.00,C,25.915300,-80.886700,FL
1821520,1821521,300240000.0,2015,2015-03-16,75,5,1.00,B,31.203436,-83.723808,GA
1808648,1808649,300224000.0,2015,2015-04-12,102,11,0.01,A,37.560000,-97.240000,KS
1843489,1843490,300268000.0,2015,2015-05-12,132,9,0.01,A,37.370000,-96.200000,KS
1791352,1791353,300203000.0,2015,2015-07-13,194,1,0.20,A,42.985556,-121.941111,OR


In [151]:
states = pd.get_dummies(test['state'])
states['fod_id'] = test['fod_id']

In [152]:
test = pd.merge(test, states, how = 'outer', on = 'fod_id')
test = test.drop(columns = ['state'])

In [153]:
train = train.drop(columns = ['discovery_date'])
test = test.drop(columns = ['discovery_date'])

In [154]:
train.head()

,objectid,fod_id,fire_year,discovery_doy,stat_cause_code,fire_size,fire_size_class,latitude,longitude,AK,AL,AR,AZ,CA,CO,FL,GA,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,PR,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1740854,300079000.0,2014,276,11,1.0,B,33.354156,-95.069997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1740854,300079000.0,2014,276,11,1.0,B,33.354156,-95.069997,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1740854,300079000.0,2014,276,11,1.0,B,33.354156,-95.069997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1740854,300079000.0,2014,276,11,1.0,B,33.354156,-95.069997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1740854,300079000.0,2014,276,11,1.0,B,33.354156,-95.069997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [163]:
train.columns

Index(['objectid', 'fod_id', 'fire_year', 'discovery_doy', 'stat_cause_code',
       'fire_size', 'fire_size_class', 'latitude', 'longitude', 'AK', 'AL',
       'AR', 'AZ', 'CA', 'CO', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS',
       'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND',
       'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
      dtype='object')

In [162]:
test.columns

Index(['objectid', 'fod_id', 'fire_year', 'discovery_doy', 'stat_cause_code',
       'fire_size', 'fire_size_class', 'latitude', 'longitude', 'AK', 'AL',
       'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'IA', 'ID', 'IL',
       'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT',
       'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA',
       'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV',
       'WY'],
      dtype='object')

In [164]:
train['CT'] = np.zeros(len(train))
train['DE'] = np.zeros(len(train))

In [167]:
test['HI'] = np.zeros(len(test))
train['DC'] = np.zeros(len(train))

In [170]:
test.shape

(317444, 61)

In [171]:
x_train = train.drop(columns = ['fire_size_class', 'fire_size'])
y_train = train[['fire_size_class']]

In [172]:
x_test.head()

,objectid,fod_id,fire_year,discovery_doy,stat_cause_code,latitude,longitude,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MI,MN,MO,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,1798263,300211000.0,2015,167,13,25.9153,-80.8867,0,0,0,0,0,0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1798263,300211000.0,2015,167,13,25.9153,-80.8867,0,0,0,0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1798263,300211000.0,2015,167,13,25.9153,-80.8867,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1798263,300211000.0,2015,167,13,25.9153,-80.8867,0,0,0,0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1798263,300211000.0,2015,167,13,25.9153,-80.8867,0,0,0,0,0,0,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [175]:
x_test = test.drop(columns = ['fire_size_class', 'fire_size'])
y_test = test[['fire_size_class']]

In [176]:
from sklearn import linear_model

In [177]:
#train logistic regression model
lr = linear_model.LogisticRegression()
lr.fit(x_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [178]:
# Train multinomial logistic regression model
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(x_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [179]:
from sklearn import metrics

In [180]:
print ("Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_train, lr.predict(x_train)))
print ("Logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, lr.predict(x_test)))

print ("Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(y_train, mul_lr.predict(x_train)))
print ("Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(y_test, mul_lr.predict(x_test)))

Logistic regression Train Accuracy ::  0.4835332312848565
Logistic regression Test Accuracy ::  0.42867718400725796
Multinomial Logistic regression Train Accuracy ::  0.5260061381777094
Multinomial Logistic regression Test Accuracy ::  0.42867718400725796


In [182]:
from sklearn.ensemble import RandomForestClassifier

In [183]:
#RF
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_pred=clf.predict(x_test)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [184]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.49994959740930683
